In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import zipfile

In [15]:
earnings = pd.read_excel('data/earnings formatted.xlsx')
earnings = earnings[earnings['Date'] > dt.datetime(2020,1,1)]
earnings = earnings.drop(earnings[earnings['Ticker'] == 'BRK'].index) # drop berkshire hathaway
earnings = earnings.drop(earnings[earnings['Ticker'] == 'BF'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'UHALB'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'LGF'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'METCV'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'GTXI'].index)
earnings

,Unnamed: 0,Ticker,Quarter,Date,Time
0,0,MSFT,2024:Q4,2024-07-30,16:00
1,1,MSFT,2024:Q3,2024-04-25,16:00
2,2,MSFT,2024:Q2,2024-01-30,16:00
3,3,MSFT,2024:Q1,2023-10-24,16:04
4,4,MSFT,2023:Q4,2023-07-25,16:00
...,...,...,...,...,...
182104,182104,GTXI,2020:Q4,2021-03-11,16:01
182105,182105,GTXI,2020:Q3,2020-11-04,17:01
182106,182106,GTXI,2020:Q2,2020-08-06,16:15
182107,182107,GTXI,2020:Q1,2020-05-07,08:00


In [14]:
returns = pd.read_csv('returns.csv.zip',compression='zip')
returns = returns.drop(returns[returns['Ticker'] == 'BRK'].index)
returns = returns.drop(returns[returns['Ticker'] == 'BF'].index)
returns = returns.drop(returns[returns['Ticker'] == 'UHALB'].index)
returns = returns.drop(returns[returns['Ticker'] == 'LGF'].index)
returns = returns.drop(returns[returns['Ticker'] == 'METCV'].index)
returns = returns.drop(returns[returns['Ticker'] == 'GTXI'].index)
returns

,Date,Ticker,Return
0,2020-01-03 00:00:00+00:00,A,-0.016056
1,2020-01-06 00:00:00+00:00,A,0.002956
2,2020-01-07 00:00:00+00:00,A,0.003065
3,2020-01-08 00:00:00+00:00,A,0.009873
4,2020-01-09 00:00:00+00:00,A,0.015712
...,...,...,...
2990267,2024-09-16 00:00:00+00:00,ZYXI,-0.022086
2990268,2024-09-17 00:00:00+00:00,ZYXI,0.027603
2990269,2024-09-18 00:00:00+00:00,ZYXI,-0.001221
2990270,2024-09-19 00:00:00+00:00,ZYXI,0.018337


In [13]:
#Disclaimer: idk if this code actually works, its based off sample.ipynb code - Andy
# Took 13:20 for this to finish running for me - Andy

ticker_list = earnings['Ticker'].unique()
tickerdata_list = []

for ticker in ticker_list:
    ticker_data = yf.Ticker(ticker).history(start=dt.datetime(2020,1,1))
    ticker_data = ticker_data.reset_index()
    
    ticker_data['Close shifted'] = ticker_data['Close'].shift(1)
    ticker_data['-1d ret'] = ticker_data['Close']/ticker_data['Close shifted'] - 1
    ticker_data['1d ret'] = ticker_data['-1d ret'].shift(-1)

    ticker_data['3 Close shifted'] = ticker_data['Close'].shift(3)
    ticker_data['-3d ret'] = ticker_data['Close']/ticker_data['3 Close shifted'] - 1
    ticker_data['3d ret'] = ticker_data['-3d ret'].shift(-3)

    ticker_data['7 Close shifted'] = ticker_data['Close'].shift(7)
    ticker_data['-7d ret'] = ticker_data['Close']/ticker_data['7 Close shifted'] - 1
    ticker_data['7d ret'] = ticker_data['-7d ret'].shift(-7)

    tickerdata_list.append(ticker_data)




final_df = pd.concat(tickerdata_list, keys=ticker_list).reset_index()
final_df = final_df.rename(columns={'level_0':'Ticker'})
final_df = final_df.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'level_1'])
final_df.head()


$BF: possibly delisted; no price data found  (1d 2020-01-01 00:00:00 -> 2024-09-21)
$UHALB: possibly delisted; no timezone found
$LGF: possibly delisted; no price data found  (1d 2020-01-01 00:00:00 -> 2024-09-21)
$METCV: possibly delisted; no timezone found
$GTXI: possibly delisted; no timezone found
C:\Users\andyq\AppData\Local\Temp\ipykernel_13424\647181932.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(tickerdata_list, keys=ticker_list).reset_index()


,Ticker,level_1,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Close shifted,-1d ret,1d ret,3 Close shifted,-3d ret,3d ret,7 Close shifted,-7d ret,7d ret,Adj Close
0,MSFT,0,2020-01-02 00:00:00-05:00,152.174719,154.043596,151.743442,153.938171,22622100.0,0.0,0.0,NaN,NaN,-0.012451,NaN,NaN,-0.018926,NaN,NaN,0.016561,NaN
1,MSFT,1,2020-01-03 00:00:00-05:00,151.733915,153.296097,151.484722,152.021423,21116200.0,0.0,0.0,153.938171,-0.012451,0.002585,NaN,NaN,0.009267,NaN,NaN,0.022128,NaN
2,MSFT,2,2020-01-06 00:00:00-05:00,150.545460,152.481432,149.999165,152.414337,20813700.0,0.0,0.0,152.021423,0.002585,-0.009118,NaN,NaN,0.019242,NaN,NaN,0.026096,NaN
3,MSFT,3,2020-01-07 00:00:00-05:00,152.692310,153.027741,150.775510,151.024689,21634100.0,0.0,0.0,152.414337,-0.009118,0.015928,153.938171,-0.018926,0.023861,NaN,NaN,0.054512,NaN
4,MSFT,4,2020-01-08 00:00:00-05:00,152.318489,154.110707,151.379262,153.430237,27746500.0,0.0,0.0,151.024689,0.015928,0.012493,152.021423,0.009267,0.019926,NaN,NaN,0.043788,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981708,WLLBW,1012,2024-09-16 00:00:00-04:00,10.490000,13.800000,10.490000,11.250000,7800.0,0.0,0.0,9.590000,0.173097,-0.065778,9.400000,0.196809,0.066667,12.76,-0.118339,NaN,NaN
2981709,WLLBW,1013,2024-09-17 00:00:00-04:00,10.510000,10.510000,10.510000,10.510000,2500.0,0.0,0.0,11.250000,-0.065778,0.019029,10.500000,0.000952,0.141770,12.14,-0.134267,NaN,NaN
2981710,WLLBW,1014,2024-09-18 00:00:00-04:00,10.320000,10.800000,10.320000,10.710000,7200.0,0.0,0.0,10.510000,0.019029,0.120448,9.590000,0.116788,NaN,13.80,-0.223913,NaN,NaN
2981711,WLLBW,1015,2024-09-19 00:00:00-04:00,12.000000,12.490000,12.000000,12.000000,6800.0,0.0,0.0,10.710000,0.120448,0.000000,11.250000,0.066667,NaN,10.75,0.116279,NaN,NaN
